In [1]:
import pandas as pd
import numpy as np

In [3]:
brightest_gal = pd.read_csv("data_files/brightest_galaxy.csv")
mag_calc = pd.read_csv("data_files/magnitude_calculated.csv")
pantheon_catalog = pd.read_csv("data_files/unmatched_pantheon.csv")

In [ ]:
data_dict = []
flags_m = 0
flags_i = 0
prev_key = -1
for i in range(len(brightest_gal)):
    for j in range(len(mag_calc)):
        if brightest_gal['SNID'][i] == mag_calc['SNID'][j]:

            
            key = brightest_gal['Galaxy_Key'][i]
            if key != prev_key:

                SNID = brightest_gal['SNID'][i]
                RA = brightest_gal['RA_Galaxy'][i]
                Dec = brightest_gal['Dec_Galaxy'][i]
                index = np.where(SNID == pantheon_catalog['SNID'])
                z = pantheon_catalog['zhel'][index[0][0]]

                H0 = 69.6
                distance_mpc = (z * 3*(10**8)) / H0
                d_pc = distance_mpc * 1e3
                app_mag = mag_calc['Zeropoint AB'][j] + brightest_gal['Mag'][i]
                abs_mag = app_mag - 5 * np.log10(d_pc/10)

                i_mag = brightest_gal['Mag'][i]

                flag_mag = False 
                flag_i = False

                if (abs_mag > -12):
                    flag_mag = True
                    flags_m += 1

                if (i_mag == 99.0):
                    flag_i = True
                    flags_i += 1

                info = {
                    "SNID": SNID,
                    "File_key": brightest_gal['File_key'][i],
                    "Galaxy_key": key,
                    "Galaxy_RA": RA,
                    "Galaxy_Dec": Dec,
                    "Zeropoint": mag_calc['Zeropoint AB'][j],
                    "Instrumental_Mag": i_mag,
                    "Apparent_Magnitude": app_mag,
                    "Absolute_Magnitude": abs_mag,
                    "Flag_mag": flag_mag,
                    "Flag_instrum": flag_i
                    }
                prev_key = brightest_gal['Galaxy_Key'][i]
                data_dict.append(info)
                

In [36]:
output_csv = "corrected_mags.csv"

df = pd.DataFrame(data_dict)
df.to_csv(output_csv, index=False)
print(f"CSV exported: {output_csv}")

CSV exported: corrected_mags.csv


In [37]:
print(flags_m)
print(flags_i)

148
25
